---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [452]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import re

In [454]:
def date_sorter():
    
    def getfirst3charofmonth(dateObj):
        splitValues = dateObj.split()
        for index, val in enumerate(splitValues):
            if re.search('[A-Za-z]+', val):
                splitValues[index] = val[:3]
        return " ".join(splitValues)

    def validdate(dateVal):
        isvalidDate = True
        try:
            parse(dateVal)
        except ValueError:
            isvalidDate = False
        return isvalidDate


    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.DataFrame(doc, columns=['text'])
    regex1WithfourdigitYear = r'\d{1,2}[/-]+\d{1,2}[/-]\d{4}'
    df['extractedstring1'] = df['text'].str.findall(regex1WithfourdigitYear)
    df['isextractedstring1valid'] = df['text'].str.findall(regex1WithfourdigitYear).str.len()

    df1 = df[df['isextractedstring1valid'] == 0]
    regex2WithtwodigitYear = r'\d{1,2}[/-]+\d{1,2}[/-]\d{2}'
    df1['extractedstring2'] = df1['text'].str.findall(regex2WithtwodigitYear)
    df1['isextractedstring2valid'] = df1['text'].str.findall(regex2WithtwodigitYear).str.len()
    columns = ['extractedstring2', 'isextractedstring2valid']
    df1 = df1[columns]
    df = df.join(df1, lsuffix="_df", rsuffix="_df1")

    df2 = df[(df['isextractedstring1valid'] == 0) & (df['isextractedstring2valid'] == 0)]
    regex3 = r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z,.]* (?:\d{1,2}, )?\d{2,4}'
    df2['extractedstring3'] = df2['text'].str.findall(regex3)
    df2['isextractedstring3valid'] = df2['text'].str.findall(regex3).str.len()
    columns = ['extractedstring3', 'isextractedstring3valid']
    df2 = df2[columns]
    df = df.join(df2, lsuffix="_df", rsuffix="_df2")

    df3 = df[(df['isextractedstring1valid'] == 0) & (df['isextractedstring2valid'] == 0) & (df['isextractedstring3valid'] == 0)]
    regex4 =  r'(?:\d{1,2}[/-]+\d{4})+'
    df3['extractedstring4'] = df3['text'].str.findall(regex4)
    df3['isextractedstring4valid'] = df3['text'].str.findall(regex4).str.len()
    columns = ['extractedstring4', 'isextractedstring4valid']
    df3 = df3[columns]
    df = df.join(df3, lsuffix="_df", rsuffix="_df3")

    df4 = df[(df['isextractedstring1valid'] == 0) & (df['isextractedstring2valid'] == 0) 
         & (df['isextractedstring3valid'] == 0) & (df['isextractedstring4valid'] == 0)]
    regex5 = r'\d{4}'
    df4['extractedstring5'] = df4['text'].str.findall(regex5)
    df4['isextractedstring5valid'] = df4['text'].str.findall(regex5).str.len()
    columns = ['extractedstring5', 'isextractedstring5valid']
    df4 = df4[columns]
    df = df.join(df4, lsuffix="_df", rsuffix="_df4")

    ext1dict = dict(df[df['isextractedstring1valid'] == 1]['extractedstring1'])
    for key, val in ext1dict.items():
        df.loc[key, 'date'] = parse(val[0].strip())

    ext2dict = dict(df[df['isextractedstring2valid'] >= 1]['extractedstring2'])
    for key, val in ext2dict.items():
        df.loc[key,'date'] = parse([item for item in val if validdate(item)][0])
    #    print([d for d in val if validdate(d)])
    #    df.loc[key,'date'] = parse(val[0].strip())

    ext3dict = dict(df[df['isextractedstring3valid'] == 1]['extractedstring3'])
    for key, val in ext3dict.items():
        if(len(val[0].strip().split()) > 2):
            #print(parse(getfirst3charofmonth(val[0].strip())))
            df.loc[key,'date'] = parse(getfirst3charofmonth(val[0].strip()))
        else:
            val[0] = "1 " + val[0]
            #print(parse(getfirst3charofmonth(val[0].strip())))
            df.loc[key,'date'] = parse(getfirst3charofmonth(val[0].strip()))

    ext4dict = dict(df[df['isextractedstring4valid'] == 1]['extractedstring4'])
    for key, val in ext4dict.items():
        #val[0] = "1/" + val[0]
        df.loc[key,'date'] =  parse("1/" + val[0].strip())
        #print(key, parse("1/" + val[0].strip()))

    ext5dict = dict(df[df['isextractedstring5valid'] == 1]['extractedstring5'])
    for key, val in ext5dict.items():
        #val[0] = "1/" + val[0]
        df.loc[key,'date'] =  parse("1/1/" + val[0].strip())
        #print(key, parse("1/1/" + val[0].strip()))

    df['Year'] = df['date'].apply(lambda x: x.year)
    dfsorted = df.sort_values('Year', axis=0)
    return pd.Series(dfsorted.index.values,np.arange(500))